In [1]:
# 数据清洗——数据转换

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## 移除重复数据

DataFrame中出现重复行有多种原因。下面就是一个例子：

In [3]:
data = DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的 `duplicated` 方法返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）：

In [4]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有一个与此相关的 `drop_duplicates` 方法，它会返回一个DataFrame，重复的行会被删除：

In [5]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我们还有一列值，且只希望根据k1列过滤重复项：

In [6]:
data.drop_duplicates(['k1'])

,k1,k2
0,one,1
1,two,1


In [7]:
data.duplicated(['k1'])

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

`duplicated`和`drop_duplicates`默认保留的是第一个出现的值组合。传入`keep='last'`则保留最后一个：

In [8]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
6,two,4


## 利用函数或映射进行数据转换

对于许多数据集，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：

In [9]:
data = DataFrame({
    'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：

In [10]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Series的map方法可以接受一个函数或含有映射关系的字典型对象，但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。
因此，我们还需要使用Series的str.lower方法，将各个值转换为小写：

In [11]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [12]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以传入一个能够完成全部这些工作的函数:

In [13]:
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


## 替换值

利用`fillna`方法填充缺失数据可以看做值替换的一种特殊情况。
前面已经看到，`map`可用于修改对象的数据子集，而`replace`则提供了一种实现该功能的更简单、更灵活的方式。我们来看看下面这个Series：

In [14]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999这个值可能是一个表示缺失数据的标记值。
要将其替换为pandas能够理解的NA值，我们可以利用`replace`来产生一个新的Series（除非传入`inplace=True`）：

In [15]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：

In [16]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

要让每个值有不同的替换值，可以传递一个替换列表：

In [17]:
data.replace([-999, -1000], [np.nan, 1000])

0       1.0
1       NaN
2       2.0
3       NaN
4    1000.0
5       3.0
dtype: float64

传入的参数也可以是字典：

In [18]:
data.replace({-999: np.nan, -1000: 1000})

0       1.0
1       NaN
2       2.0
3       NaN
4    1000.0
5       3.0
dtype: float64

## 重命名轴索引

跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。
轴还可以被就地修改，而无需新建一个数据结构。接下来看看下面这个简单的例子：

In [19]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


跟Series一样，轴索引也有一个map方法：

In [22]:
data.index = data.index.map(lambda x: x.upper())
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename：

In [25]:
data.rename(index=str.title, columns=str.upper)

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


特别说明一下，rename可以结合字典型对象实现对部分轴标签的更新：

In [26]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'one': 'first'})

,first,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


rename可以实现复制DataFrame并对其索引和列标签进行赋值。如果希望就地修改某个数据集，传入inplace=True即可

## 检测和过滤异常值

过滤或变换异常值（outlier）在很大程度上就是运用数组运算。来看一个含有正态分布数据的DataFrame：

In [27]:
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.001169,-0.010900,0.019922,-0.056536
std,1.017192,1.009932,1.019038,0.961886
min,-2.885197,-2.985485,-3.788740,-3.031148
25%,-0.704318,-0.709785,-0.662319,-0.708901
50%,-0.030675,-0.000416,0.010285,-0.077961
75%,0.727762,0.681345,0.723693,0.571965
max,4.614018,3.494743,3.832213,2.623277


假设你想要找出某列中绝对值大小超过3的值:

In [33]:
data[2][np.abs(data[2]) > 3]

24     3.303209
289   -3.300964
476   -3.576606
535   -3.788740
645    3.832213
Name: 2, dtype: float64

要选出全部含有“超过3或－3的值”的行，你可以在布尔型DataFrame中使用any方法：

In [38]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
24,0.515889,-2.711224,3.303209,0.482262
253,3.214285,0.277504,1.549705,0.678362
260,3.517650,2.195577,-1.069385,0.456647
271,0.384225,3.122319,-0.703981,-1.038841
289,0.590194,-2.782753,-3.300964,-0.212968
476,-0.887117,1.116310,-3.576606,0.997121
535,-2.674958,-0.856379,-3.788740,1.372667
645,0.535006,-0.973594,3.832213,-1.464116
655,1.292525,3.141566,1.481882,-1.416696
679,4.614018,1.079934,1.366720,0.853200


根据这些条件，就可以对值进行设置。下面的代码可以将值限制在区间－3到3以内：

In [39]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.003514,-0.011659,0.020453,-0.056505
std,1.008790,1.007526,1.009831,0.961790
min,-2.885197,-2.985485,-3.000000,-3.000000
25%,-0.704318,-0.709785,-0.662319,-0.708901
50%,-0.030675,-0.000416,0.010285,-0.077961
75%,0.727762,0.681345,0.723693,0.571965
max,3.000000,3.000000,3.000000,2.623277


根据数据的值是正还是负，np.sign(data)可以生成1和-1：

In [40]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,-1.0
1,1.0,-1.0,1.0,1.0
2,-1.0,-1.0,1.0,-1.0
3,-1.0,1.0,-1.0,1.0
4,-1.0,-1.0,-1.0,-1.0


## 排列和随机采样

利用`numpy.random.permutation`函数可以轻松实现对`Series`或`DataFrame`的列的排列工作（permuting，随机重排序）。
通过需要排列的轴的长度调用`permutation`，可产生一个表示新顺序的整数数组：

In [43]:
df = DataFrame(np.arange(20).reshape(5, 4))
sampler = np.random.permutation(5)
sampler

array([4, 2, 0, 3, 1])

然后就可以在基于`iloc`的索引操作或`take`函数中使用该数组了：

In [44]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7


如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用`sample`方法：

In [47]:
df.sample(3)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
2,8,9,10,11


要通过替换的方式产生样本（允许重复选择），可以传递`replace=True`到`sample`。
如果产生样本的数量大于元数量，则必须传入`replace=True`到`sample`：

In [54]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(10, replace=True)
draws

4    4
3    6
dtype: int64

In [52]:
choices


0    5
1    7
2   -1
3    6
4    4
dtype: int64